This is a program that will train a model to identify and assign hits to tracks.
Written by Daniel Zurawski & Keshav Kapoor for Fermilab Summer 2017 internship.

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

The below code is borrowed from a DS&HEP tutorial.
It is used to graph the histories after fitting a model.

In [ ]:
def show_losses( histories ):
    plt.figure(figsize=(10,10))
    #plt.ylim(bottom=0)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Error by Epoch')
    colors=[]
    do_acc=False
    for label,loss in histories:
        color = tuple([0.1, 0.1, 0.1])
        colors.append(color)
        l = label
        vl= label+" validation"
        if 'acc' in loss.history:
            l+=' (acc %2.4f)'% (loss.history['acc'][-1])
            do_acc = True
        if 'val_acc' in loss.history:
            vl+=' (acc %2.4f)'% (loss.history['val_acc'][-1])
            do_acc = True
        plt.plot(loss.history['loss'], label=l, color=color)
        if 'val_loss' in loss.history:
            plt.plot(loss.history['val_loss'], lw=2, ls='dashed', label=vl, color=color)


    plt.legend()
    plt.yscale('log')
    plt.show()
    if not do_acc: return
    plt.figure(figsize=(10,10))
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    for i,(label,loss) in enumerate(histories):
        color = colors[i]
        if 'acc' in loss.history:
            plt.plot(loss.history['acc'], lw=2, label=label+" accuracy", color=color)
        if 'val_acc' in loss.history:
            plt.plot(loss.history['val_acc'], lw=2, ls='dashed', label=label+" validation accuracy", color=color)
    plt.legend(loc='lower right')
    plt.show()

Here, we define a LinearTracker class.
This class is used to load input and output from a .csv file in the correct format for training the model.

In [ ]:
class LinearTracker():
    """ An object that classifies particles to tracks after an event. """    
    def __init__(self, dataframe, model=None):
        """ Initialize a LinearTracker.
            @param dataframe - pd.DataFrame - used to pick tracks from.
                The headers should contain: ("id", "z", "r", "phi").
            @param model - keras model - A network model that the tracker will
                use to classify particles.
            @return Nothing
        """
        self.model     = model     # keras model to figure out tracks.
        self.dataframe = dataframe # pandas.DataFrame for picking tracks.
        self.input     = None      # input to train model on.
        self.output    = None      # output to train model on.
    # END function __init__
    
    def load_data(self, num_events,
                  tracks_per_event, track_size, noise_per_event):
        """ Load input and output data from this object's dataframe.
            @param num_events - int - The number of events to generate.
            @param tracks_per_event - int - The number of tracks per event.
            @param track_size - int - The number of hits per track.
            @param noise_per_event - int - The number of hits with no track.
            @return Nothing
                However, self.input and self.output become numpy arrays.
                self.input is collection of hits of shape:
                    (num_events, hits_per_event, 3)
                self.output is list of probability matrices of shape:
                    (num_events, hits_per_event, tracks_per_event)
        """
        hits_per_event = (track_size * tracks_per_event) + noise_per_event
        data   = self.dataframe[["id", "r", "phi", "z"]].drop_duplicates()
        groups = data.groupby("id")
        valids = groups.filter(lambda track: len(track) == track_size)
        bads   = groups.filter(lambda track: len(track) != track_size)
        labels = ["phi", "r", "z"]
        
        # Populate input and output with data.
        self.input  = np.zeros((num_events, hits_per_event, len(labels)))
        self.output = np.zeros((num_events, hits_per_event, tracks_per_event))
        for n in range(num_events):
            # Retrieve the hits within this event.
            sample = random.sample(list(valids.groupby("id")), tracks_per_event)
            tracks = [track[1] for track in sample] # Make it not a tuple.
            noise  = bads.sample(noise_per_event)
            hits   = pd.concat(tracks + [noise])
            hits.sort_values(labels, inplace=True)
            
            # Populate this event's inputs.
            self.input[n, :] = hits[labels].values
            
            # Define a mapping from track ID to probability matrix column.
            T2I = dict()
            for t, track_ID in enumerate([s[0] for s in sample]):
                T2I[track_ID] = t
            
            # Populate this event's outputs.
            for t, track_ID in enumerate(hits["id"]):
                index = T2I.get(track_ID)
                if index is not None:
                    self.output[n, t, index] = 1
    # END FUNCTION load_data
# END CLASS LinearTracker

Below is how to create a LinearTracker and how to load data into it. It is important to note that after construction, a LinearTracker must call its load_data() function with user specifications for how data should be loaded.

If you get a ValueError describing how the population is not large enough for the sample, then that means that the data
loaded in from the .csv file does not contain enough tracks of size 'track_size'. Try to either load in a larger
population or change the 'track_size' variable to a different positive integer.

In [ ]:
filename  = ('linear_data_5k.csv')
dataframe = pd.read_csv(filename)
tracker   = LinearTracker(dataframe)

Load the data into the input and output member variables of LinearTracker.

In [ ]:
np.random.seed(7)
tracker.load_data(num_events=30, tracks_per_event=5, track_size=4, noise_per_event=5)
print("Ding! All done.")

Let's take a look at the input and output training data.

In [ ]:
from IPython.display import display,HTML

def multi_column_df_display(list_dfs, cols=2):
    """ Code by David Medenjak responding to StackOverflow question found here:
        https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
        Displays a list of dataframes in IPython as a table with cols number of columns.
    """
    html_table = "<table style='width:100%; border:0px'>{content}</table>"
    html_row = "<tr style='border:0px'>{content}</tr>"
    html_cell = "<td style='width:{width}%;vertical-align:top;border:0px'>{{content}}</td>"
    html_cell = html_cell.format(width=100/cols)

    cells = [ html_cell.format(content=df.to_html()) for df in list_dfs ]
    cells += (cols - (len(list_dfs)%cols)) * [html_cell.format(content="")] # pad
    rows = [ html_row.format(content="".join(cells[i:i+cols])) for i in range(0,len(cells),cols)]
    display(HTML(html_table.format(content="".join(rows))))

input_cols  = ["phi", "r", "z"]
output_cols = ["T{}".format(i) for i in range(tracker.output.shape[2])]

input_frames  = [pd.DataFrame(data=matrix, columns=input_cols)  for matrix in tracker.input]
output_frames = [pd.DataFrame(data=matrix, columns=output_cols) for matrix in tracker.output]

df_list = []
for i in range(max(len(input_frames), len(output_frames))):    
    df_list.append(input_frames[i])
    df_list.append(output_frames[i])

multi_column_df_display(df_list)

Let's now try to load a model into our tracker.

In [ ]:
import keras
from keras.layers import Dense, Activation, LSTM, Dropout, Flatten
from keras.models import Sequential

In [ ]:
input_shape = tracker.input[0].shape # Shape of an event.
output_shape = tracker.output[0][0].shape[0] # Number of tracks per event
kinit = 'uniform' # kernel_initializer
act   = 'relu' # activation

tracker.model = Sequential()
tracker.model.add(Dense(100, input_shape=input_shape, kernel_initializer=kinit, activation=act))
#tracker.model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
#tracker.model.add(Dense(25, kernel_initializer=kinit, activation=act))
tracker.model.add(Dense(output_shape, kernel_initializer=kinit, activation=act))
tracker.model.add(Activation('softmax'))
tracker.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
tracker.model.summary()

In [ ]:
batch_size = 32
hist = tracker.model.fit(tracker.input, tracker.output, epochs=100, batch_size=batch_size,
                         verbose=0, validation_split=0.2,
                         callbacks=[keras.callbacks.ModelCheckpoint(filepath='simple.h5', verbose=0)])
print("Ding! All done.")

It's time to graph the history of the nueral network.

In [ ]:
score, acc = tracker.model.evaluate(tracker.input, tracker.output, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
show_losses([("categorical cross entropy", hist)])